Pare esse exercício serão disponibilizados três arquivos: ***amazonas.csv*** que contém dados históricos de desflorestamento nos municipios do Amazonas no ano de 2014, ***estados.csv*** que contém uma lista com o nome, uf e codigo dos estados brasileiros e ***municipios.csv*** que contém o nome de todos os municípios brasileiros e suas respectivas localizações no globo (longitude e latitude).

O objetivo do exercício é gerar um relatório sobre o desflorestamento nos municípios do Amazonas com a visualização dos dados sobre o mapa do Brasil.
Para isso será necessário relacionar ambas as três tabelas de forma que seja associado a posição geográfica (longitude e latitude) dos municípios aos dados de desflorestamento.


In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#biblioteca para plotar no mapa
import mplleaflet

In [2]:
# Aditional packages
import os

In [4]:
# Important directories and other constants
DATA_DIR = 'data'
AMAZONAS = 'amazonas.csv'
STATES = 'estados.csv'
CITIES = 'municipios.csv'

In [6]:
#começaremos importando todas as tabelas
df_amazonas = pd.read_csv(os.path.join(DATA_DIR, AMAZONAS))
df_states = pd.read_csv(os.path.join(DATA_DIR, STATES))
df_cities = pd.read_csv(os.path.join(DATA_DIR, CITIES))

Vamos da uma olhada no jeito de cada tabela

In [7]:
df_amazonas.head()

,Municípios,Agricultura Anual,Área não Observada,Área Urbana,Desflorestamento 2014,Floresta,Hidrografia,Mineração,Mosaico de Ocupações,Não Floresta,Outros,Pasto com Solo Exposto,Pasto Limpo,Pasto Sujo,Reflorestamento,Regeneração com Pasto,Vegetação Secundária,Total Geral
0,Alvaraes,NaN,0.07,1.90,2.75,"5,415.56",133.87,NaN,101.08,43.26,5.36,NaN,2.92,1.02,NaN,5.13,198.55,"5,911.46"
1,Amatura,NaN,NaN,2.05,0.36,"4,220.22",209.41,NaN,15.87,216.55,3.76,NaN,4.03,NaN,NaN,11.44,71.74,"4,755.42"
2,Anama,NaN,0.86,0.89,0.44,"2,001.85",229.30,NaN,6.90,34.81,13.11,NaN,2.02,0.87,NaN,NaN,160.61,"2,451.66"
3,Anori,NaN,0.15,2.77,0.59,"4,646.53",436.07,NaN,17.12,636.01,4.70,NaN,8.05,0.52,NaN,NaN,41.98,"5,794.49"
4,Apui,NaN,6.71,5.91,80.19,"51,396.43",213.50,NaN,8.76,622.87,7.18,NaN,812.61,329.85,NaN,208.13,546.18,"54,238.32"


In [8]:
df_states.head()

,codigo_uf,uf,nome
0,11,RO,Rondônia
1,12,AC,Acre
2,13,AM,Amazonas
3,14,RR,Roraima
4,15,PA,Pará


In [9]:
df_cities.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31
2,5200100,Abadiânia,-16.19700,-48.7057,0,52
3,3100203,Abaeté,-19.15510,-45.4444,0,31
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15


Apos carregar os dados devemos associar as tabelas de estados e municípios e selecionar apenas os municípios do Amazonas

Note que nos dados provenientes da tabela ***municipios.csv*** o nome dos municípios estão todos acentuados, no entanto na tabela amazonas os nomes estão sem acento. Logo é necessario remover os acentos dos nomes da tabela ***municipios.csv***.

In [12]:
from unicodedata import normalize
# use esta funçao para remover acentos
def remove_acentos_python3(txt):
    return str(normalize('NFKD', txt).encode('ASCII', 'ignore'), 'utf-8')
def remove_acentos_python2(txt):
    return normalize('NFKD', txt.decode('utf-8')).encode('ASCII', 'ignore')

# ex
print(remove_acentos_python3("Olá!")) #change to python3, the one I'm using

Ola!


In [13]:
df_cities['nome'] = df_cities['nome'].apply(remove_acentos_python3)

In [14]:
df_cities.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf
0,5200050,Abadia de Goias,-16.75730,-49.4412,0,52
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31
2,5200100,Abadiania,-16.19700,-48.7057,0,52
3,3100203,Abaete,-19.15510,-45.4444,0,31
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15


Em seguida relacione as tabelas municipios e amazonas.

In [20]:
# rename column "name", because we already have a "name" column in df_cities
df_states.rename(columns={'nome': 'estado'}, inplace=True)
df_states.head()

,codigo_uf,uf,estado
0,11,RO,Rondônia
1,12,AC,Acre
2,13,AM,Amazonas
3,14,RR,Roraima
4,15,PA,Pará


In [21]:
df_brazil = df_cities.merge(df_states, how='left', on=['codigo_uf'])

In [22]:
print('Dimensões antigas:', df_cities.shape, 'Dimensões novas:', df_cities_full.shape)
df_brazil.head()

Dimensões antigas: (5570, 6) Dimensões novas: (5570, 8)


,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,uf,estado
0,5200050,Abadia de Goias,-16.75730,-49.4412,0,52,GO,Goiás
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,MG,Minas Gerais
2,5200100,Abadiania,-16.19700,-48.7057,0,52,GO,Goiás
3,3100203,Abaete,-19.15510,-45.4444,0,31,MG,Minas Gerais
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,PA,Pará


### Questão


Plote no mapa do Brasil indicadores  de desflorestamento para todos os municipios do amazonas.
Os indicadores devem ser relativos ao tamanho da floresta de cada município dando destaque aos municipios que com desflorestamento acima da média.

#### Dica

O desflorestamento relativo é dado pelas colunas ***Desflorestamento 2014*** / ***Floresta*** da tabela ***amazonas.csv***.
Note que a coluna ***Floresta*** não é numérica e contém ',' separando os milhares, sendo necessário remover a ',' de todas as entradas e logo após converter os valores para float.

Podem ser usadas todas as dicas relevantes sobre apresentação de figuras  nos textos *Making Data Meaningful* e *Ten Simple Rules for Better Figures*.

Para plotar sobre o mapa, pode-se usar a funções normais do matplotlib e ao fim chamar a função **mplleaflet.display()**.

In [131]:
#calcula as informações do desflorestamento


In [23]:
#codigo de plot
plt.figure(figsize=(8,8))

plt.plot([1], [1], 'o')

#transforma o plot em localizações no mapa
mplleaflet.display()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/display.py:694: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


### Questão


Segundo os dados, a mineração nos municípios tem alguma relação com o desflorestamento acima da média?
Mostre suas conclusões por tabelas e plots no mapa.

In [133]:
#calcula as relaçoes entre desflorestamento e mineração


In [134]:
# conclusoes sobre mineração e desflorestamento


In [24]:
#codigo de plot
plt.figure(figsize=(8,8))



#transforma o plot em localizações no mapa
mplleaflet.display()